## [캐글 필사하기] 타이타닉 for beginner

본 커널은 [@subinium 님의 자료](http://www.kaggle.com/code/subinium/subinium-tutorial-titanic-beginner/notebook)를 참고하여 필사한 자료입니다. 감사합니다 수빈님🙇🏻‍♂️

### 1. 라이브러리 불러오기

우선 코드를 작성하기에 앞서 기초적으로 필요한 라이브러리를 불러옵니다.

In [3]:
# 데이터 분석 관련
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

# 데이터 시각화 관련
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid') # matplotlib 스타일

# 그래프 출력에 필요한 IPython 명령어
%matplotlib inline

# Scikit-Learn의 다양한 머신러닝 모듈을 불러오기
# 분류 알고리즘 중에서 로지스틱 회귀, SVC & LinearSVC, 랜덤포레스트, K-최근접이웃 알고리즘을 사용할 예정
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

## 2. 데이터 읽기

Kaggle 또는 데이터 분석에서 가장 많이 사용되는 파일 형식은 `csv` 파일입니다.

코드로 데이터를 읽는 방법은 다양한 방법이 있지만, 그 중에서도 가장 유용한 것은 `pd.read_csv`로 읽는 방법입니다.

In [4]:
# 데이터 가져오기
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")

# 데이터 미리보기
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


데이터의 정보는 `info` 메서드로 확인할 수 있습니다.

훈련 데이터와 테스트 데이터를 확인해보도록 하겠습니다.

In [5]:
train_df.info()
print()
print('-' * 20)
print()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB

--------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null  

위 결과에서 각각의 데이터 개수는 891개, 418개인 것을 확인할 수 있습니다.

특성(feature)는 각각 12개, 11개인데, 그 이유는 훈련 데이터의 생존 여부(Survived)를 알고 있기 때문입니다.

여기서 주의깊게 봐야할 부분은 다음과 같습니다.
- 각 데이터는 빈 부분이 있는가?
    - 빈 부분이 있다면, `drop` 할 것인가 or `default` 값으로 채워 넣을 것인가
    - `Cabin`, `Age`, `Embarked` 세 항목에 주의
- 데이터는 float64로 변환할 수 있는가?
    - 아니라면, 범주형 데이터로 만들 수 있는가

필요 없는 부분이라고 생각되는 부분을 지웁니다. 여기서는 `PassengerId`, `Name` 그리고 `Ticket`을 지웁니다. `Name`과 `Ticket`에서 가져올 수 있는 데이터는 없기 때문입니다.

하지만, 이 문제에서 결과물은 `PassengerId`, `Survived` 요소가 필요하므로 훈련 데이터에서만 삭제합니다.

In [6]:
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
test_df = test_df.drop(['Name', 'Ticket'], axis=1)

## 3. 데이터 하나하나 처리하기

이제 남은 데이터 종류는 다음과 같습니다.
1. Pclass
2. Sex
3. Age
4. SibSp
5. Parch
6. Fare
7. Cabin
8. Embarked

순서대로 보도록 하겠습니다.

### 3.1 Pclass

Pclass는 1등석, 2등석, 3등석과 같은 정보인 서수형(순서형) 데이터입니다. 처음 확인 시에 데이터가 비어있지 않은 것을 확인할 수 있었습니다.

In [7]:
train_df['Pclass'].value_counts()

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

1, 2, 3은 정수이니, 그냥 실수로만 바꾸면 되지 않을까 생각할 수 있습니다. 하지만 1, 2, 3 등급은 경우에 따라 다를 수 있지만 연속적인 정보가 아니며, 각 차이 또한 균등하지 않습니다. 그렇기에 범주형(category) 데이터로 인식하고 인코딩해야합니다. (비슷한 예시로 영화 별점 등이 있습니다.)

이 데이터는 범주형 데이터이므로 **one-hot-encoding**을 `pd.get_dummies()` 메서드로 인코딩합시다.

In [8]:
pclass_train_dummies = pd.get_dummies(train_df['Pclass'])
pclass_test_dummies = pd.get_dummies(test_df['Pclass'])

pclass_train_dummies.columns = ['First', 'Second', 'Third']
pclass_test_dummies.columns = ['First', 'Second', 'Third']

# inplace = True : 기존 데이터프레임에 변경된 설정으로 덮어쓰겠다는 의미. 즉, 원본을 변경할지 여부
train_df.drop(['Pclass'], axis=1, inplace=True)
test_df.drop(['Pclass'], axis=1, inplace=True)

train_df = train_df.join(pclass_train_dummies)
test_df = test_df.join(pclass_test_dummies)

이렇게 Pclass의 원본을 날리고, 범주형 데이터로 데이터를 변환했습니다.

### 3.2 Sex

Sex는 성별입니다. 남과 여로 나뉘므로 이 또한 **one-hot-encoding**을 진행해봅시다.

In [9]:
sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])

sex_train_dummies.columns = ['Female', 'Male']
sex_test_dummies.columns = ['Female', 'Male']

train_df.drop(['Sex'], axis=1, inplace=True)
test_df.drop(['Sex'], axis=1, inplace=True)

train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

### 3.3 Age

나이는 연속형 데이터이므로, 큰 처리가 필요없습니다. (카테고리화를 하여 일부 알고리즘에 더 유용한 결과를 만들 수 있습니다.) 하지만, 일부 NaN 데이터가 있으니 이를 채울 수 있는 방법에 대해서 생각해봅시다.

1. 랜덤
2. 평균값
3. 중간값
4. 데이터 버리기

저는 평균값으로 채우도록 하겠습니다. 데이터의 통일성을 갖기 위해 훈련 데이터셋의 평균값으로 훈련, 테스트 데이터셋을 채우겠습니다.

In [10]:
train_df["Age"].fillna(train_df["Age"].mean(), inplace=True)
test_df["Age"].fillna(train_df["Age"].mean(), inplace=True)

### 3.4 SibSp & Panch

형제 자매와 부모님은 가족으로 함께 처리할 수 있습니다. 하지만 마찬가지로 바꿀 필요는 없습니다.

### 3.5 Fare

Fare은 탑승료입니다. 우선 빈 부분을 `fillna` 메서드로 채우겠습니다.

저는 데이터 누락이 아닌 무단 탑승이라 생각하고 0으로 입력하겠습니다.

In [11]:
test_df["Fare"].fillna(0, inplace=True)

### 3.6 Cabin

Cabin은 객실입니다. NaN이 대부분인 데이터이므로 버립시다. 이 데이터를 살리는 것은 너무 어려운 일입니다.

In [12]:
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

### 3.7 Embarked

Embarked는 탑승 항구를 의미합니다. 우선 데이터를 확인해보겠습니다.

In [13]:
train_df['Embarked'].value_counts()
test_df['Embarked'].value_counts()

Embarked
S    270
C    102
Q     46
Name: count, dtype: int64

S가 대다수이고 train_df의 전체 데이터 개수는 891인데, 'Embarked'에 해당하는 train_df의 데이터 개수 합이 889(644 + 168 + 77)입니다. 따라서, 빈 부분은 S로 채우고 시작합시다.

In [14]:
train_df["Embarked"].fillna('S', inplace=True)
test_df["Embarked"].fillna('S', inplace=True)

'S', 'C', 'Q'로 나뉘므로 이 또한 **one-hot-encoding**을 진행해봅시다.

In [15]:
embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

embarked_train_dummies.columns = ['S', 'C', 'Q']
embarked_test_dummies.columns = ['S', 'C', 'Q']

train_df.drop(['Embarked'], axis=1, inplace=True)
test_df.drop(['Embarked'], axis=1, inplace=True)

train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_test_dummies)

## 4. 데이터 나누기

이제 학습용 데이터를 위해 데이터를 나누어야합니다.

`(정보, 생존 여부)`와 같은 형태를 위하여 다음과 같이 데이터를 나눕니다.

In [16]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test = test_df.drop("PassengerId", axis=1).copy()

## 5. 머신러닝 알고리즘 적용하기

이제 로지스틱 회귀,SVC, 랜덤 포레스트, K-최근접이웃 알고리즘을 각각 적용해봅시다

In [17]:
# Logistic Regression
## Binary한 데이터가 주어질 때는 Linear Regression을 사용하는 것보다는 Logistic Regression을 사용하는 것이 훨씬 정확도가 높다.

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, Y_train)
                       
logreg.score(X_train, Y_train)

0.8047138047138047

In [18]:
# Support Verctor Machines
## SVM은 패턴 인식, 분류 그리고 회귀 분석 등의 머신러닝 알고리즘이다.

svc = SVC()
svc.fit(X_train, Y_train)

svc.score(X_train, Y_train)

0.6868686868686869

In [19]:
# Random Forests
## 기존 Bagging의 이점을 살리고 변수를 랜덤으로 선택하는 과정을 추가함으로써 개별 나무들의 상관성을 줄여 예측력을 향상한 앙상블 모형

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)

random_forest.score(X_train, Y_train)

0.9820426487093153

In [20]:
# K-Nearest Neighbor
## 주변의 가장 가까운 K개의 데이터를 보고 데이터가 속할 그룹을 판단하는 알고리즘

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, Y_train)

knn.score(X_train, Y_train)

0.835016835016835

## 6. 제출용 파일 만들기

**Random Forest**가 가장 좋은 결과를 내는 것을 알 수 있습니다. 그 결과로 submission 파일을 만들어 제출해봅시다.

In [21]:
# Random Forests

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)

submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": Y_pred
})
submission.to_csv('titanic.csv', index=False)